# Import modules

In [ ]:
import os
from tinydb import TinyDB
import pandas as pd
import time
from DashPykpi.kpistats import KpiStats, GitURLs, GraphKPIs

# Create DB

Need to cover the repos assoicated with the below github user accounts

✔ Indicates the repos are covered by rc-softdev-admin account...
 
1. https://github.com/UCL-RITS   ✔
1. https://github.com/astro-informatics
1. https://github.com/bempp
1. https://github.com/DCPROGS
1. https://github.com/OPTIMET
1. https://github.com/UCL-Biochemical-Engineering
1. https://github.com/UCL-CSS
1. https://github.com/UCLProgrammingHub
1. https://github.com/UCL-HIC
1. https://github.com/EIT-team
1. https://github.com/Astrophysics-UCL
1. https://github.com/Bahler-Lab
1. https://github.com/CCPPETMR
1. https://github.com/Euclid-OULE3
1. https://github.com/FieldingChemistryUCL
1. https://github.com/PancreaticProject
1. https://github.com/ShapsUcl
1. https://github.com/msslsolar
1. https://github.com/mssl-plasma-group
1. https://github.com/UCL-ShippingGroup

In [ ]:
# or... use a list of URLS fetched from the GitURLs class
url_fetch = GitURLs()
urls = url_fetch.urls
print("Retrieved {0} urls.".format(len(urls)))

In [ ]:
# These projects don't exist anymore and break get_repo_stats
urls.remove('https://github.com/UCL/ucl')
urls.remove('https://github.com/UCL-RITS/ucl-rits')
urls

In [ ]:
# needed = [
#     "https://github.com/UCL-RITS",
#     "https://github.com/astro-informatics",
#     "https://github.com/bempp",
#     "https://github.com/DCPROGS",
#     "https://github.com/OPTIMET",
#     "https://github.com/UCL-Biochemical-Engineering",
#     "https://github.com/UCL-CSS",
#     "https://github.com/UCLProgrammingHub",
#     "https://github.com/UCL-HIC",
#     "https://github.com/EIT-team",
#     "https://github.com/Astrophysics-UCL",
#     "https://github.com/Bahler-Lab",
#     "https://github.com/CCPPETMR",
#     "https://github.com/Euclid-OULE3",
#     "https://github.com/FieldingChemistryUCL",
#     "https://github.com/PancreaticProject",
#     "https://github.com/ShapsUcl",
#     "https://github.com/msslsolar",
#     "https://github.com/mssl-plasma-group",
#     "https://github.com/UCL-ShippingGroup"]
    
# need_accounts = [n.split('/')[3] for n in needed]
# got_accounts = [url.split('/')[-2] for url in urls]

In [ ]:
# from github3 import GitHub
# fn = open("secret_key")
# g = GitHub(token=fn.read().split()[0])

# for thing in g.iter_user_repos(login='benlaken'):
#     print(thing)
    
# for need in need_accounts:
#     if not need in got_accounts:
#         print("Missing {0} repos ".format(need, need in got_accounts))
#         print("    Attempting to access - will work if public")
        
#         for thing in g.iter_user_repos(login=need):
#             print("    {0}".format(thing))

In [ ]:
#for url in urls:
#    user_str, repo_str = url.split('/')[-2:]
#    print(url, user_str, repo_str)

In [ ]:
# Run the code to generate a dababase if needed...
test = KpiStats(urls=urls)
test.work(verbose=False, debug=False, status=True)

# Plotting section

In [ ]:
db = TinyDB('tinydb_for_KPI.json')

In [ ]:
df = pd.DataFrame(db.all())

In [ ]:
df

In [ ]:
#for n in df['repo_owner']:
#    print(n)

In [ ]:
running = 0
num_repos = 0
tmp_hold = {}
for n, weekly in enumerate(df['weekly_commits']):
    if sum(weekly) > 1:
        tmp_hold[df['repo_name'][n]] = weekly
        running += sum(weekly)
        num_repos += 1
print("{0:3,} commits, in {1} active repos (out of {2} total repos), during past 52 weeks".format(
        running, num_repos, len(df)))

In [ ]:
import numpy as np

In [ ]:
tmp = []
for n, weekly in enumerate(df['weekly_commits']):
    if sum(weekly) > 1:
        tmp.append(weekly)

tmp = np.array(tmp)
tmp = tmp.sum(axis=0)
all_weekly_commits = {"All repos": tmp}

In [ ]:
width=4
data = tmp

result = data[:(data.size // width) * width].reshape(-1, width).sum(axis=1)
result
print(52/4)

In [ ]:
all_weekly_commits

## Stacked area chart

In [ ]:
#from bokeh.charts import Area, show, output_notebook, defaults
#import numpy as np
from bokeh.charts import show, output_notebook
output_notebook()


In [ ]:
bk = GraphKPIs()
show(bk.weekly_activity(per_repo=False, verbose=True, bin=4))

In [ ]:
def weekly_activity(per_repo=False, width = 800, height = 400, give_script_div=False, verbose=False):
    """Create a stacked area plot covering the past 52 weeks of commit acvitity.
    
    Plot in the notebook (assuming a TinyDB file exists).
    
    :Example:
    
    >>>from DashPykpi.kpistats import GraphKPIs
    >>>show(GraphKPIs.weekly_activity())
    """
    #df = self.df
    defaults.width = width
    defaults.height = height
    if per_repo:
        running = 0
        num_repos = 0
        tmp_hold = {}
        for n, weekly in enumerate(df['weekly_commits']):
            if sum(weekly) > 1:
                tmp_hold[df['repo_name'][n]] = weekly
                running += sum(weekly)
                num_repos += 1
        if verbose:
            print("{0:3,} commits, in {1} active repos (out of {2} total repos), during past 52 weeks".format(
                    running, num_repos, len(df)))
        area = Area(tmp_hold, title="Commits to all repos", legend=None,
                     stack=True, xlabel='weeks since present ', ylabel='number of commits')
        
        if give_script_div: 
            # This is incase you want to add the graphics directly to a HTML template file
            script, div = components(area)
            return script, div
        else:
            return(area) 
    if not per_repo:
        tmp = []
        for n, weekly in enumerate(df['weekly_commits']):
            if sum(weekly) > 1:
                tmp.append(weekly)
        tmp = np.array(tmp)
        tmp = tmp.sum(axis=0)
        all_weekly_commits = {"All repos": tmp}
        area = Area(all_weekly_commits, title="Commits to repos", legend=None,
                     stack=True, xlabel='weeks since present ', ylabel='number of commits')
        if give_script_div: 
            # This is incase you want to add the graphics directly to a HTML template file
            script, div = components(area)
            return script, div
        else:
            return(area)

In [ ]:
show(weekly_activity(per_repo=False, verbose=True))

In [ ]:
p = bk.weekly_activity(per_repo=False, verbose=True, bin=4)

In [ ]:
p.xaxis.

In [ ]:
import numpy as np
a = np.array([1,2,3,4])

In [ ]:
test = []
for item in a:
    test.append([item] * 4)


In [ ]:
np.array(test).flatten()